In [10]:
from Bio import Phylo
import copy
import random

def get_neighbors(tree):
        """Get all neighbor trees of the given tree (PRIVATE).
        Currently only for binary rooted trees.
        """
        # make child to parent dict
        parents = {}
        for clade in tree.find_clades():
            if clade != tree.root:
                node_path = tree.get_path(clade)
                # cannot get the parent if the parent is root. Bug?
                if len(node_path) == 1:
                    parents[clade] = tree.root
                else:
                    parents[clade] = node_path[-2]
        neighbors = []
        root_childs = []
        for clade in tree.get_nonterminals(order="level"):
            if clade == tree.root:
                left = clade.clades[0]
                right = clade.clades[1]
                root_childs.append(left)
                root_childs.append(right)
                if not left.is_terminal() and not right.is_terminal():
                    # make changes around the left_left clade
                    # left_left = left.clades[0]
                    left_right = left.clades[1]
                    right_left = right.clades[0]
                    right_right = right.clades[1]
                    # neighbor 1 (left_left + right_right)
                    del left.clades[1]
                    del right.clades[1]
                    left.clades.append(right_right)
                    right.clades.append(left_right)
                    temp_tree = copy.deepcopy(tree)
                    neighbors.append(temp_tree)
                    # neighbor 2 (left_left + right_left)
                    del left.clades[1]
                    del right.clades[0]
                    left.clades.append(right_left)
                    right.clades.append(right_right)
                    temp_tree = copy.deepcopy(tree)
                    neighbors.append(temp_tree)
                    # change back (left_left + left_right)
                    del left.clades[1]
                    del right.clades[0]
                    left.clades.append(left_right)
                    right.clades.insert(0, right_left)
            elif clade in root_childs:
                # skip root child
                continue
            else:
                # method for other clades
                # make changes around the parent clade
                left = clade.clades[0]
                right = clade.clades[1]
                parent = parents[clade]
                if clade == parent.clades[0]:
                    sister = parent.clades[1]
                    # neighbor 1 (parent + right)
                    del parent.clades[1]
                    del clade.clades[1]
                    parent.clades.append(right)
                    clade.clades.append(sister)
                    temp_tree = copy.deepcopy(tree)
                    neighbors.append(temp_tree)
                    # neighbor 2 (parent + left)
                    del parent.clades[1]
                    del clade.clades[0]
                    parent.clades.append(left)
                    clade.clades.append(right)
                    temp_tree = copy.deepcopy(tree)
                    neighbors.append(temp_tree)
                    # change back (parent + sister)
                    del parent.clades[1]
                    del clade.clades[0]
                    parent.clades.append(sister)
                    clade.clades.insert(0, left)
                else:
                    sister = parent.clades[0]
                    # neighbor 1 (parent + right)
                    del parent.clades[0]
                    del clade.clades[1]
                    parent.clades.insert(0, right)
                    clade.clades.append(sister)
                    temp_tree = copy.deepcopy(tree)
                    neighbors.append(temp_tree)
                    # neighbor 2 (parent + left)
                    del parent.clades[0]
                    del clade.clades[0]
                    parent.clades.insert(0, left)
                    clade.clades.append(right)
                    temp_tree = copy.deepcopy(tree)
                    neighbors.append(temp_tree)
                    # change back (parent + sister)
                    del parent.clades[0]
                    del clade.clades[0]
                    parent.clades.insert(0, sister)
                    clade.clades.insert(0, left)
        return neighbors
    


def get_more_neighbors(tree, level, degree):
    trees = [[tree]]
    cur_level = 0
    while cur_level < level:
        new_trees = []
        for tree in random.sample(trees[-1], min(degree, len(trees[-1]))):
            new_trees += get_neighbors(tree)
        trees.append(new_trees)
        cur_level+=1
    return [tree for sublist in trees for tree in sublist]
tree = Phylo.read("/home/luise/master_thesis/scripts/data/trees/cognate_ie_compatible.tree", format="newick")
#Phylo.write(get_neighbors(tree), "/home/luise/master_thesis/scripts/data/trees/cognate_neighbors.trees", format="newick")
Phylo.write(get_more_neighbors(tree, 90, 1), "/home/luise/master_thesis/scripts/data/trees/cognate_neighbors_90.trees", format="newick")

7741

In [11]:
from ete3 import Tree
def distance_sample(tree_set_path, ref_tree_path, out_file_path):
    ref_tree  = Tree(ref_tree_path)
    dist_dict = {}
    lines = open(tree_set_path, 'r').readlines()
    for line in lines:
        t = Tree(line)
        rf, max_rf, common_leaves, parts_t1, parts_t2,discard_t1, discart_t2 = ref_tree.robinson_foulds(t, unrooted_trees = True)
        if not rf in dist_dict.keys():
            dist_dict[rf] = t
    out_file = open(out_file_path, "w+")
    print(dist_dict.keys())
    for (k, t) in dist_dict.items():
        out_file.write(t.write()+"\n")
            
distance_sample("/home/luise/master_thesis/scripts/data/trees/cognate_neighbors_90.trees", 
                "/home/luise/master_thesis/scripts/data/trees/cognate_ie_compatible.tree", 
               "/home/luise/master_thesis/scripts/data/trees/cognate_neighbors_90_sample.trees")



dict_keys([0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32, 34, 36, 38, 40, 42, 44, 46, 48, 50, 52, 54, 56, 58, 60, 62, 64, 66, 68, 70, 72])
